In [1]:
import hashlib
import json
import pandas as pd
import numpy as np
import urllib
import re
import urllib.request as urllib2
import datetime
from time import sleep
import time
# from .chuli import dealpd, col_mul
import talib
import operator
import psycopg2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header

In [ ]:
# 计算两天close涨幅 T1,T2
def zf(data):
    resdata = []
    for r in data.itertuples():
        time = getattr(r, 'time')
        open = getattr(r, 'open')
        high = getattr(r, 'high')
        low = getattr(r, 'low')
        close = getattr(r, 'close')
        resdata.append([time, open, high, low, close])
    res = [0]
    for i in range(1, len(resdata)):
        T = (float(resdata[i][4]) - float(resdata[i - 1][4])) / float(resdata[i - 1][4]) * 100
        res.append(T)
    return res


# 计算TP和值
def myTP(T1, T2, Q):
    res = [0]

    for i in range(len(T1)):
        t = T1[i] * (1 - Q / 100) + T2[i] * (Q / 100)
        res.append(t)
    return res


# 计算TP差值
def myTM(T1, T2, Q):
    res = [0]
    for i in range(len(T1)):
        t = T1[i] * (1 - Q / 100) - T2[i] * (Q / 100)
        res.append(t)
    return res


# 计算和值累加 或 差值累加
def mySUM(TP):
    res = [0]
    for i in TP:
        res.append(res[-1] + i)
    return res


# 求指定周期均值，参数1数据列表，参数2周期，返回单个数值
def myAverage(lists, tp):
    if len(lists) > tp:
        return np.mean(lists[-tp:])
    else:
        return np.mean(lists)


# 判断趋势
def pdqs(lists):
    count = -1
    while lists[count] == lists[count - 1]:
        count -= 1
    if lists[count] > lists[count - 1]:
        return "趋势向上"
    elif lists[count] < lists[count - 1]:
        return "趋势向下"


# 判断交叉
# def pdjx(ddlist, eelist):
#     for i in range(len(ddlist)):
#         if ddlist[-i]:
#             return "金叉"
#         elif eelist[-i]:
#             return "死叉"
#         else:
#             return '没有交叉'
# 判断交叉
def pdjx(ddlist, eelist):
    for i in range(len(ddlist)):
        if ddlist[-i]:
            return "金叉"
        elif eelist[-i]:
            return "死叉"
        else:
            return '没有交叉'


# 计算结果 参数：合约列表，合约数据字典，短周期，长周期，权重2,截止日期
def comres(symlist, datadic, S, L, Q2, starttime, endtime):
    qrsortdic = {symlist[0]: 0}
    for s in range(len(symlist) - 1):
        data0 = datadic[symlist[0]]
        data1 = datadic[symlist[s + 1]]
        pd.to_datetime(data0.ix[:, 0])  # 转换为时间格式
        pd.to_datetime(data1.ix[:, 0])  # 转换为时间格式

        # timestr = '2018-10-19 00:00:00'

        data0 = data0[(data0.ix[:, 0] <= endtime) & (data0.ix[:, 0] >= starttime)]
        data1 = data1[(data1.ix[:, 0] <= endtime) & (data1.ix[:, 0] >= starttime)]
        print('筛选后')
        print(len(data0), len(data1))
        T1 = zf(data0)  # 第一腿涨幅
        T2 = zf(data1)  # 第二腿涨幅

        # TP = myTP(T1, T2, Q1)  # 相对和
        TM = myTM(T1, T2, Q2)  # 相对差

        # TPSUM = mySUM(TP)  # 相对和累加
        TMSUM = mySUM(TM)  # 相对差累加

        TMS = np.array(TMSUM)
        # TPS = np.array(TPSUM)

        MA1 = talib.MA(TMS, S)  # 相对差短周期均线
        MA2 = talib.MA(TMS, L)  # 相对差长周期均线

        # MAS = talib.MA(TPS, S)
        # MAL = talib.MA(TPS, L)
        qrsortdic[symlist[s + 1]] = MA1[-1] - MA2[-1]  # 正数说明强度大于data0
    sorted_res = sorted(qrsortdic.items(), key=operator.itemgetter(1), reverse=False)
    return sorted_res


def get_data(conn, symlist, period):
    """获取数据"""
    datadic = {}
    for sym in symlist:
        s = sym
        if sym == 'BTC':
            sym = 'xbtusd'
        else:
            sym = sym.lower() + 'u18z18'
        sqllasttime = " SELECT time, open, high, low, close, volume from " + sym + "_" + period + " order by time"
        df = pd.read_sql(sqllasttime, con=conn)
        datadic[s] = df
    return datadic


def send_email(username, password, receiver, text):
    """发送邮件"""
    smtpserver = 'smtp.qq.com'
    sender = username

    subject = '强弱排名'
    subject = Header(subject, 'utf-8').encode()

    msg = MIMEMultipart('mixed')
    msg['Subject'] = subject
    msg['From'] = username
    # msg['To'] = 'XXX@126.com'
    # 收件人为多个收件人,通过join将列表转换为以;为间隔的字符串
    msg['To'] = ";".join(receiver)
    # msg['Date']='2012-3-16'

    text_plain = MIMEText(text, 'plain', 'utf-8')
    msg.attach(text_plain)

    # 发送邮件
    smtp = smtplib.SMTP()
    smtp.connect('smtp.qq.com')
    # 我们用set_debuglevel(1)就可以打印出和SMTP服务器交互的所有信息。
    # smtp.set_debuglevel(1)
    smtp.login(username, password)
    smtp.sendmail(sender, receiver, msg.as_string())
    smtp.quit()


def strtotimestamp(timestr):
    """
    时间字符串转unix时间戳
    :param str: 时间字符串
    :return: unix时间戳，str类型
    """
    dt = datetime.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
    timestamp = time.mktime(dt.timetuple())
    return str(int(timestamp))


def timestamptostr(timestamp):
    """
    unix时间戳转时间字符串
    :param stamp:unix时间戳，字符串类型
    :return:时间字符串
    """
    datestr = datetime.datetime.fromtimestamp(int(timestamp))
    return datestr


def getdata(url):
    """
    爬取数据
    :param url: 链接
    :return: json数据
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    request = urllib2.Request(url=url, headers=headers)
    try:
        b_data = urllib2.urlopen(request, timeout=1000).read()
    except:
        print("获取数据出现异常")
        time.sleep(30)
        b_data = urllib2.urlopen(request, timeout=1000).read()
    # print(b_data)
    # b_data = b_data.decode()
    # print(type(b_data))
    # print(b_data)
    # json解析
    b_data = json.loads(b_data)
    # print(type(b_data))
    # print(b_data[0]['timestamp'])
    # print(b_data)
    # for i in b_data:
    #     print(i)
    return b_data


def savadata(b_data, binSize, symbol):
    """
    保存数据
    :param b_data: 数据
    :return: 下一次的url
    """
    conn = psycopg2.connect(database='bitmex', user="postgres", password="hello", host="127.0.0.1", port="5432")

    cur = conn.cursor()
    # symbol = symbol.lower()
    print(b_data)
    sql = "INSERT INTO " + symbol + "_" + binSize + " (timestamp , time , open , high, low, close , volume)  VALUES (\'{}\',\'{}\',{},{},{},{},{});"
    uzsql = "INSERT INTO " + symbol[
                             0:3] + "u18z18_" + binSize + " (timestamp , time , open , high, low, close , volume)  VALUES (\'{}\',\'{}\',{},{},{},{},{});"
    for v in b_data:
        bitmextime = v['timestamp']
        time = re.sub('T', ' ', re.sub('\.000Z', '', bitmextime))
        time = (datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(days=-1)).strftime(
            '%Y-%m-%d %H:%M:%S')
        timestamp = strtotimestamp(str(time))
        open = v['open']
        high = v['high']
        low = v['low']
        close = v['close']
        volume = v['volume']
        sqlg = sql.format(str(timestamp), str(time), open, high, low, close, volume)
        uzsqlg = uzsql.format(str(timestamp), str(time), open, high, low, close, volume)
        print(sqlg)
        try:
            cur.execute(sqlg)
            conn.commit()  # 事务提交
        except Exception as e:
            print(e)
            conn.commit()  # 事务提交 当出现错误时，try中commit将不会被执行导致以后命令忽略
        try:
            if symbol == 'ETHUSD' or symbol == 'ethusd':
                continue
            cur.execute(uzsqlg)
            conn.commit()  # 事务提交
        except Exception as e:
            print(e)
            conn.commit()  # 事务提交 当出现错误时，try中commit将不会被执行导致以后命令忽略
    cur.close()
    conn.close()
    lasttime = b_data[-1]['timestamp']
    print(lasttime)
    url = 'https://www.bitmex.com/api/v1/trade/bucketed?{}&{}&{}&{}&{}&{}'
    url = url.format('binSize=' + binSize, 'partial=false', 'symbol=' + symbol, 'count=500', 'reverse=false',
                     'startTime=' + lasttime)
    return url


def getlasttime(sym, binSize):
    """
    从数据库获取最后一条数据的时间
    :param sym: 合约
    :param binSize: 周期
    :return: 最后时间
    """
    conn = psycopg2.connect(database="bitmex", user="postgres", password="hello", host="127.0.0.1",
                            port="5432")
    cur = conn.cursor()
    sqllasttime = " SELECT time from " + sym + "_" + binSize + " order by id desc limit 1"
    try:
        cur.execute(sqllasttime)
        rows = cur.fetchall()  # all rows in table
        ltime = rows[0][0]
        ltime = (datetime.datetime.strptime(ltime, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(days=1)).strftime(
            '%Y-%m-%d %H:%M:%S')
    except Exception as e:
        print(e)
        ltime = '2018-11-20 00:00:00'
    lt = re.sub(' ', 'T', re.sub(':(?P<min>.*):(?P<sec>.*)', ':\g<min>:\g<sec>.000Z', ltime))
    conn.commit()
    cur.close()
    conn.close()
    return str(lt)


def sleeptime(timestr):
    now_time = datetime.datetime.now()
    nowhms = time.strftime("%H:%M:%S")
    # 目标时间与今天还未到达
    if nowhms < timestr:
        now_year = now_time.year
        now_month = now_time.month
        now_day = now_time.day
        aimtime = datetime.datetime.strptime(str(now_year) + "-" + str(now_month) + "-" + str(now_day) + " " + timestr,
                                             "%Y-%m-%d %H:%M:%S")
        sleeptime = (aimtime - now_time).total_seconds()
        print('当前时间', now_time)
        print('目标时间', aimtime)
        print('睡眠时间', sleeptime)
        time.sleep(sleeptime)
    else:
        # 目标时间今天已过
        aimtime = now_time + datetime.timedelta(days=+1)
        aim_year = aimtime.year
        aim_month = aimtime.month
        aim_day = aimtime.day
        aimtime = datetime.datetime.strptime(str(aim_year) + "-" + str(aim_month) + "-" + str(aim_day) + " " + timestr,
                                             "%Y-%m-%d %H:%M:%S")
        sleeptime = (aimtime - now_time).total_seconds()
        print('当前时间', now_time)
        print('目标时间', aimtime)
        print('睡眠时间', sleeptime)
        time.sleep(sleeptime)


def main_com(symlist):
    pass


# M18,U18,Z18
# XBTUSD，ETHUSD，ETHU18，XBTUSD，BCHU18，BCHU18，LTCU18，XRPU18,TRXU18
# 1m 差479m，5m 差475m
# spd_symlist = ['XBTUSD', 'ETHUSD', 'ADAZ18', 'BCHZ18', 'EOSZ18', 'LTCZ18', 'TRXZ18', 'XRPZ18']
# spd_symlist = ['XBTU18', 'ETHU18', 'ADAU18', 'BCHU18', 'EOSU18', 'LTCU18', 'TRXU18', 'XRPU18']

# spd_symlist = ['XBTUSD', 'ETHUSD','XBTZ18', 'ETHZ18', 'ADAZ18', 'BCHZ18', 'EOSZ18', 'LTCZ18', 'TRXZ18', 'XRPZ18']
spd_symlist = ['XBTUSD', 'ETHZ18', 'ADAZ18', 'BCHZ18', 'EOSZ18', 'LTCZ18', 'TRXZ18', 'XRPZ18']

while True:
    acttime = '08:05:00'
    #sleeptime(acttime)
    for s in spd_symlist:

        # f_time = str(int(time.time()))
        # path = "./deal/"
        binSize = "1d"
        symbol = s
        # Z系列09-21开始
        startTime = getlasttime(symbol, binSize)
        # startTime = "2018-11-01T00:00:00.000Z"
        print(startTime)
        # if symbol == 'XBTZ18':
        #     startTime = "2018-07-02T00:00:00.000Z"

        # conn = psycopg2.connect(database='bitmex', user="postgres", password="hello", host="127.0.0.1", port="5432")

        url = 'https://www.bitmex.com/api/v1/trade/bucketed?{}&{}&{}&{}&{}&{}'

        url = url.format('binSize=' + binSize, 'partial=false', 'symbol=' + symbol, 'count=500', 'reverse=false',
                         'startTime=' + startTime)
        print('开始获取' + symbol + "合约数据！")
        print('开始url', url)

        count = 1
        while True:
            print('获取第' + str(count) + 'url数据', url)
            b_data = getdata(url)
            if len(b_data) >= 1:
                # print(b_data)
                url = savadata(b_data, binSize, symbol)
                sleep(1)
                print('break')
                break
            else:
                break
            count += 1
            if (count % 10) == 0:
                print('休眠10秒')
                sleep(5)
            if (count % 30) == 0:
                print('继续休眠10秒')
                sleep(10)
            if (count % 50) == 0:
                print('再休眠10秒')
                sleep(10)
            if (count % 100) == 0:
                print('再休眠10秒')
                sleep(10)
        print(symbol + '合约数据获取完毕！')
    print('所有合约数据获取完成')

    username = '499428970@qq.com'
    password = 'oklvhkylrbtqbjja'
    sender = '499428970@qq.com'
    # receiver='XXX@126.com'
    # 收件人为多个收件人
    #receiver = ['304381005@qq.com','1340815253@qq.com']
    receiver = ['1340815253@qq.com']
    conn = psycopg2.connect(database="bitmex", user="postgres", password="hello", host="127.0.0.1", port="5432")

    # 合约列表
    symlist = ['ADA', 'TRX', 'BCH', 'BTC', 'LTC', 'XRP', 'ETH', 'EOS']
    symlist2 = ['ADA', 'TRX', 'BCH', 'LTC', 'XRP', 'ETH', 'EOS']
    binSize = '1d'
    starttime = '2018-06-29 00:00:00'
    endtime = '2019-11-02 00:00:00'

    Q1 = 50  # 权重1
    Q2 = 50  # 权重2 相对差
    S = 3  # 短周期
    L = 5  # 长周期w
    datadic1 = get_data(conn, symlist, binSize)
    # print(datadic)
    r1 = comres(symlist, datadic1, S, L, Q2, starttime, endtime)
    #print(r1)

    datadic2 = get_data(conn, symlist2, binSize)
    # print(datadic)
    r2 = comres(symlist2, datadic2, S, L, Q2, starttime, endtime)
    #print(r2)

    now_time = str((datetime.datetime.now() + datetime.timedelta(days=-1)).strftime('%Y-%m-%d'))
    stext1 = now_time+'强弱对比8品种排名\n'
    for i in range(len(r1)):
        stext1 = stext1 + r1[i][0] + ":" + str(7 - i) + "\n"
    #print(stext1)

    stext2 = now_time+'强弱对比7品种排名\n'
    for i in range(len(r2)):
        stext2 = stext2 + r2[i][0] + ":" + str(6 - i) + "\n"
    #print(stext2)

    restext = stext1+stext2
    print(restext)
    send_email(username, password, receiver, restext)
    conn.close()
    break
